Did you know?


**A person can be turned into a diamond!!**

Want to live on for eternity? With the advancement of lab-grown diamond technology, some companies are now offering to turn the ashes of a deceased love one into a diamond gem that can worn as jewelry by those they left behind. Swiss-based Algordanza has been providing memorial diamonds since 2003, and now operates in 30 different countries.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

from PIL import Image
sns.set(style="ticks", color_codes=True)
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [ ]:
diamond=pd.read_csv('diamonds.csv')
diamond.head(2)

In [ ]:
Image.open('Anglo-DiamondAnatomy_03.JPG')

In [ ]:
diamond.drop('Unnamed: 0',axis=1,inplace=True)

print('Numnber of records in dataset are {} and Features are {}.  '.format(*diamond.shape))
print("\nAny missing sample in set:",diamond.isnull().values.any())

In [ ]:
diamond.describe()

In [ ]:
print("Categorical column's details :")
diamond.describe(include=['O'])

In [ ]:
plt.figure(figsize=(20,5))
for i,feature in enumerate(diamond.select_dtypes(include='O').columns.tolist(),1):
    plt.subplot(1,3,i)
    diamond[feature].value_counts(normalize=True).plot.bar()
    plt.title(feature)

plt.figure(figsize=(20,5))
for i,feature in enumerate(diamond.select_dtypes(include='O').columns.tolist(),1):
    plt.subplot(1,3,i)
    sns.boxplot(x=feature,y='price',data=diamond,palette='husl')
plt.show()


 **CUT** 
* Most of the cut present in dataset are of 'Ideal' type with around 40% but median price for them are lowest.
* 'Fair' cut is least found and hence has high price range comparitively.

**COLOR** 
* G,E,F are easily available colors and economical too.
* I,J color type will cost you more bucks :)

**CLARITY** 
* SI1 and VS2 are the most often found clarities in diamonds but need to pay more for SI1 clarity than VS2.

**Note :**
* Price exception are for all cases irrespective of cut,color,clarity.


In [ ]:
cut_clarity=pd.pivot_table(diamond, values='price', columns='cut', index='clarity', aggfunc='median')
cut_color=pd.pivot_table(diamond, values='price', columns='cut', index='color', aggfunc='median')
clarity_color=pd.pivot_table(diamond, values='price', columns='clarity', index='color', aggfunc='median')

print("Combination of Color,Cut,Clarity with median price ")
plt.figure(figsize=(20,5))
plt.subplot(131)
sns.heatmap(cut_clarity,cmap='Blues')

plt.subplot(132)
sns.heatmap(cut_color,cmap='Greys')

plt.subplot(133)
sns.heatmap(clarity_color,cmap='Blues')
plt.show()

* Combination of Premium cut,SI1 clarity and H,I,J color makes diamond costly.
* IF cut generally doesn't attract more money, but its combination with D color were sold foe higher median price.

In [ ]:
plt.figure(figsize=(10,5))
diamond.corr()['price'].sort_values()[:-1].plot.barh()
plt.title('Order of dependence of price on Numerical Features')
plt.xlabel('Correaltion coefficient with Price')
plt.ylabel('Feature')
plt.show()

* Carat and x,y,z feature are deciding factor for price calculation. 
* Table and depth has very little effect in price of diamond.

In [ ]:
list=['carat', 'depth', 'table', 'x', 'y', 'z']
plt.figure(figsize=(15,10))
for i,feature in enumerate(list,1):
    plt.subplot(2,3,i)
    sns.scatterplot(diamond[feature],diamond['price'])
    plt.title(feature)
plt.show()

* Trend of price with carat,x,y,z can be easily seen from scatterplots.
* While Depth and Table concentrated more in their particular range and have no strong pattern with price.
* Some outliers are clearly visible in scatterplots and need to be treated.

In [ ]:
list=['carat', 'depth', 'table', 'x', 'y', 'z']
plt.figure(figsize=(15,8))
for i,feature in enumerate(diamond.select_dtypes(exclude='O').columns.tolist(),1):
    plt.subplot(3,3,i)
    plt.hist(diamond[feature],bins=100)
    plt.title(feature)
plt.show()

* Carat, price and x covers a wide distribution with significant skewness.
* Certainly our numerical features are not normally distributed.

In [ ]:
sns.heatmap(diamond.corr(),annot=True)
plt.show()

* Lot of interdependency can be seen between features, as multiple features have high correlation coefficient.

## 1) Raw OLS model


In [ ]:
diamond_d=pd.get_dummies(diamond,columns=diamond.select_dtypes(include='O').columns.tolist(),drop_first=True)

X=diamond_d.drop('price',axis=1)
y=diamond.price

Xc = sm.add_constant(X)
lin_reg = sm.OLS(y,Xc).fit()
lin_reg.summary()

In [ ]:
df_results = pd.DataFrame(columns=['Description','R2','Adjusted_R2','MSE_Model','MSE_Resid'])

description = "Raw OLS"
r2=round(lin_reg.rsquared,3)
adjR2=round(lin_reg.rsquared_adj,3)
msem=round(lin_reg.mse_model,2)
mser=round(lin_reg.mse_resid,2)
df_results = pd.concat([df_results,pd.DataFrame(np.array([description,r2,adjR2,msem,mser]).reshape(1,-1),
            columns=['Description','R2','Adjusted_R2','MSE_Model','MSE_Resid'])], axis=0)

df_results

## 2) OLS model after log transformation of Price


In [ ]:
X
y_log=y.apply(np.log)

Xc = sm.add_constant(X)
lin_reg_log = sm.OLS(y_log,Xc).fit()
lin_reg_log.summary()

In [ ]:
description = "OLS + Log Price"
r2=round(lin_reg_log.rsquared,3)
adjR2=round(lin_reg_log.rsquared_adj,3)
msem=round(lin_reg_log.mse_model,2)
mser=round(lin_reg_log.mse_resid,2)
df_results = pd.concat([df_results,pd.DataFrame(np.array([description,r2,adjR2,msem,mser]).reshape(1,-1),
            columns=['Description','R2','Adjusted_R2','MSE_Model','MSE_Resid'])], axis=0)

df_results

# Checking the Asumptions of Linear Regression

For Linear Regression, we need to check if the 5 major assumptions hold.

1. No Auto correlation
2. Normality of error terms
3. Linearity of variables 
4. No Heteroscedacity
5. No strong MultiCollinearity


##### Assumption 1- No autocorrelation



Test done : Durbin- Watson Test.

- It's value ranges from 0-4. 
- If the value of Durbin- Watson is Between 0-2, it's known as Positive Autocorrelation.
- If the value ranges from 2-4, it is known as Negative autocorrelation.
- If the value is exactly 2 (generally can take range from 1.5 to 2.5), it means No Autocorrelation .
- For a good linear model, it should have low or no autocorrelation.

we can see here the values of dublin watson test (From OLS summary) : 

1.190 (1st case),

1.533 (2nd case)

Hence no autocorrelation before and after log transformation of price.
    


In [ ]:
import statsmodels.tsa.api as smt

acf = smt.graphics.plot_acf(lin_reg.resid, lags=40 , alpha=0.05)
acf.show()

From the graph below, we can easily see that there is no autocorrelation.

##### Assumption 2- Normality of Residuals

Test done : Jarque Bera test.

* The Jarque–Bera test is a goodness-of-fit test of whether sample data have the skewness and kurtosis matching a normal distribution.
* A large value for the jarque-bera test indicates non normality.
    

In [ ]:
from scipy import stats
print('JB value before log Transformation of price :',stats.jarque_bera(lin_reg.resid)[0])
print('JB value  after log Transformation of price :',stats.jarque_bera(lin_reg_log.resid)[0])

* The critical chi square value at the 5% level of significance is 5.99. 
* In both case the computed value of the JB statistic is greater than 5.99. 
* Thus we reject the null hypothesis and hence the error terms are not normally distributed.

In [ ]:
plt.figure(figsize=(15,4))
plt.subplot(121)
plt.title('Before log Transformation of price')
sns.distplot(lin_reg.resid)
plt.subplot(122)
plt.title('After log Transformation of price')
sns.distplot(lin_reg_log.resid)
plt.show()

Residuals have high kurtosis value, which makes it non linear.

##### Asssumption 3 - Linearity of residuals


Test done : Rainbow test 

* It is done to check the linearity of the residuals for a linear regression model.
* Linearity of residuals is preferred.

 $$ H_{0} : Residuals ~are~ linear $$
 $$ H_{1} : Residuals~ are ~not ~linear $$

In [ ]:
import statsmodels.api as sm
#checking 50% of fraction of data, since sample of data can be linear.
lstat1,pvalue1=sm.stats.diagnostic.linear_rainbow(res=lin_reg, frac=0.5)  
lstat2,pvalue2=sm.stats.diagnostic.linear_rainbow(res=lin_reg_log, frac=0.5)  

print('The pvalue before log Transformation of price :',round(pvalue1,3))
print('The pvalue after log Transformation of price :',round(pvalue2,3))

* As pvalue > 0.05, We failed to reject H0, Hence residuals are linear.

In [ ]:
import scipy.stats as stats
import pylab
from statsmodels.graphics.gofplots import ProbPlot

print('Probability Plot:\n 1)Before\n 2)After log Transformation of price')
plt.figure(figsize=(15,5))
plt.subplot(121)
st_residual = lin_reg.get_influence().resid_studentized_internal
stats.probplot(st_residual, dist="norm", plot = pylab)

plt.subplot(122)
st_residual = lin_reg_log.get_influence().resid_studentized_internal
stats.probplot(st_residual, dist="norm", plot = pylab)
plt.show()

* Points are symmetrically distributed around a diagonal line in both case(more after log Transformation of price).
* Hence linearity of residuals can be seen.

In [ ]:
print('The mean value of residuals should be zero.\n')
print('The mean value of residuals before log Transformation of price :',lin_reg.resid.mean())
print('The mean value of residuals after log Transformation of price :',lin_reg_log.resid.mean())

* Very much close to zero in both case, So linearity is present.

##### Assumption 4 -  Homoscedasticity_test 

Test done : Goldfeld Test

* If the variance of the residuals are symmetrically distributed across the regression line , then the data is said to homoscedastic and the residuals should be homoscedastic.
* This test is based on the hytpothesis testing where null and alternate hypothesis are:
$$ H_{0} : \sigma_{u_{i}}~is~constant~across~the~range~of~data $$

$$ H_{a} : \sigma_{u_{i}}~is~not~constant~across~the~range~of~data $$




In [ ]:
import statsmodels.stats.api as sms

pvalue1 = sms.het_goldfeldquandt(lin_reg.resid, lin_reg.model.exog)[1]
pvalue2 = sms.het_goldfeldquandt(lin_reg_log.resid, lin_reg_log.model.exog)[1]

print('The pvalue before log Transformation of price :',round(pvalue1,3))
print('The pvalue after log Transformation of price :',round(pvalue2,3))

Here, p value > 0.05 so we failed to reject null hypothesis, it is homoscedasticity distribution.

##### Assumption 5- NO  MULTI COLLINEARITY

* As, we can see from the second warning of OLS model itself that Multicollinearity is present in the dataset.
* It can also be stated by noting the high conditional number.


In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif = [variance_inflation_factor(Xc.values, i) for i in range(Xc.shape[1])]
pd.DataFrame({'vif': vif[1:]}, index=X.columns).T

* The interpretation of VIF is as follows: the square root of a given variable’s VIF shows how much larger the standard error is, compared with what it would be if that predictor were uncorrelated with the other features in the model. If no features are correlated, then all values for VIF will be 1.
* So, multicollinearity exists here as vif values are high.

##  Dropping feature with highest VIF value

In [ ]:
diamond_d2=diamond_d.drop('x',axis=1)

X2=diamond_d2.drop('price',axis=1)
y=diamond.price

Xc2 = sm.add_constant(X2)
lin_reg2 = sm.OLS(y,Xc2).fit()

description = "OLS + without X feature "
r2=round(lin_reg2.rsquared,3)
adjR2=round(lin_reg2.rsquared_adj,3)
msem=round(lin_reg2.mse_model,2)
mser=round(lin_reg2.mse_resid,2)
df_results = pd.concat([df_results,pd.DataFrame(np.array([description,r2,adjR2,msem,mser]).reshape(1,-1),
            columns=['Description','R2','Adjusted_R2','MSE_Model','MSE_Resid'])], axis=0)

df_results

## OLS + without X feature + Log Price

In [ ]:
X2
y_log

Xc3 = sm.add_constant(X2)
lin_reg3 = sm.OLS(y_log,Xc3).fit()

description = "OLS + without X feature + Log Price"
r2=round(lin_reg3.rsquared,3)
adjR2=round(lin_reg3.rsquared_adj,3)
msem=round(lin_reg3.mse_model,2)
mser=round(lin_reg3.mse_resid,2)
df_results = pd.concat([df_results,pd.DataFrame(np.array([description,r2,adjR2,msem,mser]).reshape(1,-1),
            columns=['Description','R2','Adjusted_R2','MSE_Model','MSE_Resid'])], axis=0)

df_results

Lets check VIF again.

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
pd.set_option('Display.max_columns',None)
vif = [variance_inflation_factor(Xc3.values, i) for i in range(Xc3.shape[1])]
pd.DataFrame({'vif': vif[1:]}, index=X2.columns).T

Still multicolinearity can be seen in some features, so we will be dropping those feature and will check performance of model again.

## OLS + No multicolinear Feature

In [ ]:
diamond_d4=diamond_d.drop(['z','cut_Ideal','clarity_SI1'],axis=1)

X4=diamond_d4.drop('price',axis=1)
y=diamond.price

Xc4 = sm.add_constant(X4)
lin_reg4 = sm.OLS(y,Xc4).fit()

description = "OLS + No multicolinear Feature "
r2=round(lin_reg4.rsquared,3)
adjR2=round(lin_reg4.rsquared_adj,3)
msem=round(lin_reg4.mse_model,2)
mser=round(lin_reg4.mse_resid,2)
df_results = pd.concat([df_results,pd.DataFrame(np.array([description,r2,adjR2,msem,mser]).reshape(1,-1),
            columns=['Description','R2','Adjusted_R2','MSE_Model','MSE_Resid'])], axis=0)

df_results

## OLS + No multicolinear Feature + Log Price

In [ ]:
X4
y_log

Xc5 = sm.add_constant(X4)
lin_reg5 = sm.OLS(y_log,Xc5).fit()

description = "OLS + No multicolinear Feature + Log Price"
r2=round(lin_reg5.rsquared,3)
adjR2=round(lin_reg5.rsquared_adj,3)
msem=round(lin_reg5.mse_model,2)
mser=round(lin_reg5.mse_resid,2)
df_results = pd.concat([df_results,pd.DataFrame(np.array([description,r2,adjR2,msem,mser]).reshape(1,-1),
            columns=['Description','R2','Adjusted_R2','MSE_Model','MSE_Resid'])], axis=0)

df_results

* From above results we can infer that taking log transformation on price feature is giving better results in all cases.
* R2 and AdjustedR2 values are almost similar.
* OLS model with Log of price can be considered for further studies as it is giving best result in predicting price of diamonds.


**Do Upvote if you liked my work.**

**Happy learning!!**